In [4]:
%pip install opencv-python
%pip install git+https://github.com/ageitgey/face_recognition_models
%pip install face_recognition
%pip install face_recognition_models

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 3.3 MB/s eta 0:00:12
   - -------------------------------------- 1.0/39.5 MB 2.6 MB/s eta 0:00:15
   - -------------------------------------- 1.6/39.5 MB 2.4 MB/s eta 0:00:16
   - -------------------------------------- 1.8/39.5 MB 2.2 MB/s eta 0:00:17
   -- ------------------------------------- 2.1/39.5 MB 2.1 MB/s eta 0:00:18
   -- ------------------------------------- 2.4/39.5 MB 2.1 MB/s eta 0:00:18
   -- ------------------------------------- 2.6/39.5 MB 1.9 MB/s eta 0:00:20
   -- ------------------------------------- 2.9/39.5 MB 1.9 MB/s eta 0:00:20
   --- ------------------------------------ 3.1/39.5 MB 1.7 MB/s eta 0:00:21
   --- ------------------------------------ 3.4/39.5 MB 1.7 MB/s eta 0:00:22
   --- ------------------------------------ 3.7/39.5 MB 1.6 MB/s eta 0:00:23
   --- ------

In [2]:
import cv2 as cv
import dlib
import face_recognition
import numpy as np
from PIL import Image

In [ ]:
# webcam live feed and capturing images from that
# WORKING

camera = cv.VideoCapture(0)
frame_no = 1

if not camera.isOpened():
    print('couldnt open cam')
    exit()
else:
    while True:
        ret, vid = camera.read()
        
        if not ret:
            print('couldnt fetch camera frames')
            break
        
        rgb_frame = cv.cvtColor(vid, cv.COLOR_BGR2RGB)
        
        cv.imshow('frame', rgb_frame)
        
        key = cv.waitKey(1) & 0xFF
        
        if key == ord('c'):
            
            scale_factor = 0.5  # Adjust as needed
            new_width = int(rgb_frame.shape[1] * scale_factor)
            new_height = int(rgb_frame.shape[0] * scale_factor)
            resized_image = cv.resize(rgb_frame, (new_width, new_height))
            
            cv.imwrite(f'Snapshots/frame{frame_no}.jpg', resized_image)
            frame_no += 1
            continue
        
        if key == ord('q'):
            break
    
camera.release()
cv.destroyAllWindows()

In [ ]:
# checking image dimensions and RGB status

from PIL import Image

image = Image.open("Snapshots/yeezy.jpg")

if image.mode == "RGB":
    print("Image is in RGB format")
elif image.mode == "L":
    print("Image is grayscale")
else:
    print(f"Image is in {image.mode} format")
    
img = Image.open("Snapshots/yeezy.jpg")
img = img.convert("RGB")  # Ensure no alpha channel
width = image.width
height = image.height

print('width and heigth: ', width, height)
img.save("fixed_image.jpg")


Image is in RGB format
width and heigth:  1200 800


In [ ]:
# function to detect face in image and crop the image to just the face
# face_location function error
# NOT WORKING

MODEL = 'cnn'

def face_crop(image_path):
      
    loaded_image = face_recognition.load_image_file(image_path)
         
    face_location = face_recognition.face_locations(loaded_image, model=MODEL)
    
    for i, (top, right, bottom, left) in enumerate(face_location):
        
        face_image = loaded_image[top:bottom, right:left]
        face_image_bgr = cv.cvtColor(face_image, cv.COLOR_RGB2BGR)
    
    return face_image_bgr
    
x = face_crop('fixed_image.jpg')

cv.imshow(f"hi", x)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# attempt using dlib
# NOT WORKING (same error)

detector = dlib.get_frontal_face_detector()

image = cv.imread('Snapshots/yeezy.jpg')
image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)

if image_rgb.dtype != np.uint8:
    image_rgb = (255 * (image_rgb / image_rgb.max())).astype(np.uint8)

print(f"Image Type: {image_rgb.dtype}")  # Should print 'uint8'
print(f"Image Shape: {image_rgb.shape}")

cv.imwrite("fixed_image.jpg", cv.cvtColor(image_rgb, cv.COLOR_RGB2BGR))

image = face_recognition.load_image_file('fixed_image.jpg')

faces = detector(image)